# A brief summary of the Standard Library in C++ (98)
<br>
<div style="opacity: 0.8; font-family: Consolas, Monaco, Lucida Console, Liberation Mono, DejaVu Sans Mono, Bitstream Vera Sans Mono, Courier New; font-size: 12px; font-style: italic;">
    ────────
    for more from the author, visit
    <a href="https://github.com/hazemanwer2000">github.com/hazemanwer2000</a>.
    ────────
</div>

## Table of Contents
* [Containers](#containers)
    * [Sequences](#sequences)
        * [`vector`](#vector)
        * [`list`](#list)
        * [`deque`](#deque)
    * [Sequence Adapters](#sequence-adapters)
        * [`stack`](#stack)
        * [`queue`](#queue)
        * [`priority_queue`](#priority-queue)
    * [Associative Containers](#associative-containers)
        * [`map`](#map)
        * [`multimap`](#multimap)
        * [`set`](#set)
        * [`multiset`](#multiset)
* [Algorithms and Function Objects](#algorithms-and-function-objects)
    * [Function Objects](#function-objects)
        * [Predicates](#predicates)
        * [Arithmetic Functors](#arithmetic-functors)
        * [Binders](#binders)
        * [Function Adapters](#function-adapters)
        * [Negaters](#negaters)
    * [Algorithms](#algorithms)
        * [Non-modifying Sequence Operations](#non-modifying-sequence-operations)
        * [Modifying Sequence Operations](#modifying-sequence-operations)
        * [Sorting Sequence Operations](#sorting-sequence-operations)
        * [Set Operations](#set-operations)
        * [Heap Operations](#heap-operations)
        * [Permutations](#permutations)

<hr>

The C++ standard library resides in the namespace `std`.

* Header files are included as `<header-name>`, omitting the `.h` extension. For example, `<vector>`.

* Header files from the C standard library may be included as `<c-header-name>`. For example, `<cstdio>`.

* Header files from the C standard library may be included directly into the global namespace as `<header-name.h>`. For example, `<stdio.h>`.

* User-defined header files may be included as `"header-name.h"`. For example, `"myvector.h"`.

## Containers <a class="anchor" id="containers"></a>

A *container* is an object that stores a collection of other objects. They are implemented as class templates.

Containers share some standard methods, though with no derivation from a common base. This is to enable inlining of most method calls, while still allowing for the implementation of pieces of code that deal with containers generically, and not a particular type of container, through the use of templates. This comes, ofcourse, at the expense of inflated code size.

### Sequences <a class="anchor" id="sequences"></a>

A *sequence* is an enumerated collection of objects in which repetitions are allowed and order matters.

#### `vector` <a class="anchor" id="vector"></a>

A `vector` is an array, that is able to grow dynamically and implicitly.

In [ ]:
std::vector<int> vec1;          /* 0 elements */

In [ ]:
std::vector<int> vec2(5);       /* 5 elements, copy-initialized with a default-initialized object */

In [ ]:
std::vector<int> vec3(5, 3);    /* 5 elements, copy-initialized with a passed object */

Growth involves possible re-allocation of elements, similar to how a function like `realloc` operates. Hence, growth should not occur with every append to its content. Therefore, `vector` defines both methods `size`, that returns the number of elements in the `vector`, and `capacity`, that returns the maximum number of elements that a `vector` can currently withstand without incurring a growth overhead.

*Note:* A `vector` never shrinks in capacity.

In [42]:
#include <vector>
#include <iostream>

template<class T>
void summary(std::vector<T> &vec) {
    std::cout << "Size: " << vec.size() << ", Capacity: " << vec.capacity() << '\n';
}

int main() {
    std::vector<int> vec1;
    summary(vec1);
    
    std::vector<int> vec2(5);
    summary(vec2);
}

Size: 0, Capacity: 0
Size: 5, Capacity: 5


`vector` defines the method `reserve`, that accepts the number of elements to set the capacity of the `vector` to. If the argument is less than the current capacity of the `vector`, nothing happens.

*Note:* `reserve` merely allocates memory to avoid a growth overhead later on. It does not construct any objects.

`vector` also defines the method `resize`, that accepts the number of elements to set the size of the `vector` to. The argument may be less than the current size of the `vector`.

*Note:* Unlike `reserve`, `resize` initializes memory with copy-initialized objects, copied from a default-initialized object, or an optionally passed object.

In [48]:
#include <vector>
#include <iostream>

template<class T>
void summary(std::vector<T> &vec) {
    std::cout << "Size: " << vec.size();
    std::cout << ", Capacity: " << vec.capacity();
    std::cout << ", [";
    for (int i = 0; i < vec.size(); i++) {
        std::cout << vec[i] << ", ";
    }
    std::cout << "\b\b]\n";
}

template<class T>
void print_vec(std::vector<T> &vec) {
    for (int i = 0; i < vec.size(); i++) {
        std::cout << vec[i] << ", ";
    }
    std::cout << "\b\b \n";
}

int main() {
    std::vector<int> vec(5, 1);        /* 5 elements, copy-initialized with '1' */
    summary(vec);
    
    vec.reserve(10);                   /* capacity guranteed to be at least '10' */
    summary(vec);
    
    vec.reserve(8);                    /* capacity guranteed to be at least '8' (already '10') */
    summary(vec);
    
    vec.resize(8, 2);                  /* enlarged to 8 elements, last three initialized with '2' */
    summary(vec);
    
    vec.resize(7);                     /* shrinked to 7 elements */
    summary(vec);
}

Size: 5, Capacity: 5, [1, 1, 1, 1, 1, ]
Size: 5, Capacity: 10, [1, 1, 1, 1, 1, ]
Size: 5, Capacity: 10, [1, 1, 1, 1, 1, ]
Size: 8, Capacity: 10, [1, 1, 1, 1, 1, 2, 2, 2, ]
Size: 7, Capacity: 10, [1, 1, 1, 1, 1, 2, 2, ]


`vector` defines the method `max_size`, that returns the maximum number of elements that `vector` can withstand.

In [52]:
#include <iostream>
#include <vector>

int main() {
    std::vector<float> vfloat;
    std::cout << vfloat.max_size() << '\n';
    
    std::vector<double> vdouble;
    std::cout << vdouble.max_size() << '\n';
}

2305843009213693951
1152921504606846975


*Note:* Since, usually, `double` occupies double the number of bytes of `float`, the maximum size of `vector<double>` is half the maximum size of `vector<float>`. This is because `vector` is an array-like structure, and uses pointers to provide constant element access. As the size of the offset between each pointer increases, the number of objects that can be represented by a given pointer size decreases.

`vector` defines the method `empty`, that evaluates and returns `size() == 0`, as a `bool`.

In [62]:
#include <iostream>
#include <vector>

int main() {
    std::vector<int> vec(5);
    std::cout << vec.empty() << '\n';
    
    vec.resize(0);
    std::cout << vec.empty() << '\n';
}

0
1


`vector` defines the operator `[]` for unchecked access to elements. It also defines the method `at` for checked access.

In [73]:
#include <iostream>
#include <vector>

template<class T>
void summary(std::vector<T> &vec) {
    std::cout << "Size: " << vec.size();
    std::cout << ", Capacity: " << vec.capacity();
    std::cout << ", [";
    for (int i = 0; i < vec.size(); i++) {
        std::cout << vec[i] << ", ";
    }
    std::cout << "\b\b]\n";
}

int main() {
    std::vector<int> vec(3);
    
    vec[0] = 1;
    vec[1] = 2;
    vec[2] = 3;
    vec[3] = 4;           /* Unchecked out-of-range access */
                          /*   Possible memory corruption, or segmentation-fault error */
    
    summary(vec);
}

Size: 3, Capacity: 3, [1, 2, 3, ]


In [79]:
#include <iostream>
#include <vector>

template<class T>
void summary(std::vector<T> &vec) {
    std::cout << "Size: " << vec.size();
    std::cout << ", Capacity: " << vec.capacity();
    std::cout << ", [";
    for (int i = 0; i < vec.size(); i++) {
        std::cout << vec[i] << ", ";
    }
    std::cout << "\b\b]\n";
}

int main() {
    std::vector<int> vec(3);
    
    try {
        vec.at(0) = 1;
        vec.at(1) = 2;
        vec.at(2) = 3;
        vec.at(3) = 4;           /* Checked out-of-range access */
                                 /*   'std::out_of_range' exception thrown */
    } catch (std::out_of_range &e) {
        std::cout << "[Error] Out-of-range access.\n";
    }
    
    summary(vec);
}

[Error] Out-of-range access.
Size: 3, Capacity: 3, [1, 2, 3, ]


*Note:* Since `vector` is basically an array, element access has $O(1)$ worst-case complexity.

*Note:* Naturally, the `at` method incurs a constant overhead, compared to `[]`.

`vector` defines the `front` and `back` methods, that return the first and last elements, respectively.

In [85]:
#include <iostream>
#include <vector>

int main() {
    std::vector<int> vec(5);
    
    for (int i = 0; i < vec.size(); i++) {
        vec[i] = i + 1;                      /* 1, 2, 3, ... */
    }

    std::cout << "Front: " << vec.front() << '\n';
    std::cout << "Back: " << vec.back() << '\n';
}

Front: 1
Back: 5


`vector` defines the `push_back` and `pop_back` methods.

* `push_back` accepts a passed object to add to the end of a `vector`, incrementing its size.
* `pop_back` removes the last element of a `vector`, decrementing its size, and returns `void`.

In this way, `vector` may be used as a stack, with $O(1)$ worst-case complexity of stack operations, ignoring the possible growth overhead.

In [91]:
#include <iostream>
#include <vector>

template<class T>
void summary(std::vector<T> &vec) {
    std::cout << "Size: " << vec.size();
    std::cout << ", Capacity: " << vec.capacity();
    std::cout << ", [";
    for (int i = 0; i < vec.size(); i++) {
        std::cout << vec[i] << ", ";
    }
    std::cout << "\b\b]\n";
}

int main() {
    std::vector<int> vec;
    
    vec.reserve(5);

    for (int i = 0; i < vec.capacity(); i++) {
        vec.push_back(i+1);                               /* Pushing onto stack */
    }
    
    summary(vec);
    
    std::cout << "Top of stack: " << vec.back() << '\n';    /* Peaking top-of-stack */
    
    vec.pop_back();                                       /* Popping off stack */
    
    summary(vec);
}

Size: 5, Capacity: 5, [1, 2, 3, 4, 5, ]
Top of stack: 5
Size: 4, Capacity: 5, [1, 2, 3, 4, ]


`vector` defines `begin` and `end` methods, that return iterators to the first and one after the last elements, respectively.

An *iterator* is a pointer-like object to an element. Iterators can be used to navigate containers, without knowledge of the actual type used to identify elements.

`vector` also defines the type `iterator`, that is the type of iterator it deals with.

In [107]:
#include <iostream>
#include <vector>

int main() {
    std::vector<int> vec(5);
    
    for (int i = 0; i < vec.capacity(); i++) {
        vec[i] = i + 1;
    }
    
    std::cout << *(vec.begin() + 2) << '\n';         /* equivalent to vec[2]      */
    std::cout << *(vec.end() - 1) << '\n';           /* equivalent to vec.front() */ 
}

3
5


In [117]:
#include <iostream>
#include <vector>

template<class T>
void forwardly(std::vector<T> &vec) {
    typename std::vector<T>::iterator iter = vec.begin();
    while (iter != vec.end()) {
        std::cout << *iter++ << ", ";
    }
    std::cout << "\b\b \n";
}

int main() {
    std::vector<int> vec(5);
    
    for (int i = 0; i < vec.capacity(); i++) {
        vec[i] = i + 1;
    }
    
    forwardly(vec);
}

1, 2, 3, 4, 5,  


`vector` defines `rbegin` and `rend` methods, that return reverse iterators to the last and one before the first elements, respectively.

`vector` also defines the type `reverse_iterator`, that is the type of reverse iterator it deals with.

The `base` method of a reverse iterator, returns an iterator to the next element.

In [115]:
#include <iostream>
#include <vector>

int main() {
    std::vector<int> vec(5);
    
    for (int i = 0; i < vec.capacity(); i++) {
        vec[i] = i + 1;
    }
    
    std::cout << *(vec.rbegin()) << '\n';             /* equivalent to vec.back() */
    std::cout << *(vec.rbegin() + 2) << '\n';         /* equivalent to vec[vec.size() - 1 - 2] */
    std::cout << *(vec.rend().base()) << '\n';        /* equivalent to vec.front() or *(vec.rbegin()) */
}

5
3
1


In [116]:
#include <iostream>
#include <vector>

template<class T>
void backwardly(std::vector<T> &vec) {
    typename std::vector<T>::reverse_iterator iter = vec.rbegin();
    while (iter != vec.rend()) {
        std::cout << *iter++ << ", ";
    }
    std::cout << "\b\b \n";
}

int main() {
    std::vector<int> vec(5);
    
    for (int i = 0; i < vec.capacity(); i++) {
        vec[i] = i + 1;
    }
    
    backwardly(vec);
}

5, 4, 3, 2, 1 


*Note:* Iterators in `vector` are random-access. That is, `ITERATOR + CONSTANT` is always of $O(1)$ worst-case complexity.

*Note:* Beware, with changes to a `vector`, previously defined iterators may become undefined.

*Note:* `vector` also defines types `const_iterator` and `const_reverse_iterator` for read-access only.

`vector` defines a copy constructor, that accepts a reference to the same type of `vector`.

In [124]:
#include <iostream>
#include <vector>

template<class T>
void forwardly(std::vector<T> &vec) {
    typename std::vector<T>::iterator iter = vec.begin();
    while (iter != vec.end()) {
        std::cout << *iter++ << ", ";
    }
    std::cout << "\b\b \n";
}

int main() {
    std::vector v1(5, 1);
    std::vector v2(v1);
    
    v2[0] = 3;
    
    forwardly(v1);
    forwardly(v2);
}

1, 1, 1, 1, 1,  
3, 1, 1, 1, 1,  


`vector` defines a templated constructor, that accepts two objects, defining the beginning and end (exclusive) from which to copy. These objects must match in type, and must allow for dereferencing and incremening operations.

In [134]:
#include <iostream>
#include <vector>

template<class T>
void forwardly(std::vector<T> &vec) {
    typename std::vector<T>::iterator iter = vec.begin();
    while (iter != vec.end()) {
        std::cout << *iter++ << ", ";
    }
    std::cout << "\b\b \n";
}

int main() {
    std::vector<int> v1(5);
    
    for (int i = 0; i < v1.size(); i++) {
        v1[i] = i + 1;
    }
    
    std::vector<int> v2(v1.begin() + 1, v1.end() - 1);
    
    char s[] = "HELLO";
    
    std::vector<char> v3(s, s + 5);
    
    forwardly(v1);
    forwardly(v2);
    forwardly(v3);
}

1, 2, 3, 4, 5,  
2, 3, 4,  
H, E, L, L, O,  


In addition, `vector` overloads the assignment operator, and provides two additional `assign` methods.

In [137]:
#include <iostream>
#include <vector>

template<class T>
void forwardly(std::vector<T> &vec) {
    typename std::vector<T>::iterator iter = vec.begin();
    while (iter != vec.end()) {
        std::cout << *iter++ << ", ";
    }
    std::cout << "\b\b \n";
}

int main() {
    std::vector<int> v1(5);
    
    for (int i = 0; i < v1.size(); i++) {
        v1[i] = i + 1;
    }
    
    std::vector<int> v2;
    v2 = v1;                                  /* Copies 'v1' into 'v2' (grows implicitly) */
    
    std::vector<int> v3;
    v3.assign(v1.begin() + 2, v1.end());      /* Copies from 'A' to 'B - 1' */
    
    std::vector<int> v4;
    v4.assign(5, 0);                          /* Copies '0' five times (Second parameter is non-optional) */
    
    forwardly(v1);
    forwardly(v2);
    forwardly(v3);
    forwardly(v4);
}

1, 2, 3, 4, 5,  
1, 2, 3, 4, 5,  
3, 4, 5,  
0, 0, 0, 0, 0,  


`vector` defines the method `insert`, to insert elements into a vector, in $O(n)$ worst-case complexity.

In [142]:
#include <iostream>
#include <vector>

template<class T>
void forwardly(std::vector<T> &vec) {
    typename std::vector<T>::iterator iter = vec.begin();
    while (iter != vec.end()) {
        std::cout << *iter++ << ", ";
    }
    std::cout << "\b\b \n";
}

int main() {
    std::vector<int> v1(5);
    
    for (int i = 0; i < v1.size(); i++) {
        v1[i] = i + 1;
    }
    
    std::vector<int> v2(v1);
    v2.insert(v2.begin(), 0);                             /* Insert '0' at beginning */
    
    std::vector<int> v3(v1);
    v3.insert(v3.end(), 3, 0);                            /* Insert three '0' at end */
    
    std::vector<int> v4(4, 0);
    v4.insert(v4.begin() + 2, v1.begin(), v1.end());      /* Insert at index '2', from 'A' to 'B - 1' */
                                                          /* 'A' and 'B' must allow deref. and inc.   */

    forwardly(v2);
    forwardly(v3);
    forwardly(v4);
}

0, 1, 2, 3, 4, 5,  
1, 2, 3, 4, 5, 0, 0, 0,  
0, 0, 1, 2, 3, 4, 5, 0, 0,  


`vector` defines the method `erase`, to remove elements from a vector, in $O(n)$ worse-case complexity.

In [150]:
#include <iostream>
#include <vector>

template<class T>
void forwardly(std::vector<T> &vec) {
    typename std::vector<T>::iterator iter = vec.begin();
    while (iter != vec.end()) {
        std::cout << *iter++ << ", ";
    }
    std::cout << "\b\b \n";
}

int main() {
    std::vector<int> v(5);
    
    for (int i = 0; i < v.size(); i++) {
        v[i] = i + 1;
    }
    
    forwardly(v);
    
    v.erase(v.begin());                      /* Remove at iterator 'A' */
    forwardly(v);
    
    v.erase(v.begin(), v.begin() + 2);       /* Remove at iterator 'A' to 'B - 1' */
    forwardly(v);
}

1, 2, 3, 4, 5,  
2, 3, 4, 5,  
4, 5,  


`vector` defines the method `clear`, that removes all elements from a vector, with $O(n)$ complexity.

In [153]:
#include <iostream>
#include <vector>

template<class T>
void summary(std::vector<T> &vec) {
    std::cout << "Size: " << vec.size() << ", Capacity: " << vec.capacity() << '\n';
}

int main() {
    std::vector<int> v(5);
    
    summary(v);
    
    v.clear();
    
    summary(v);
}

Size: 5, Capacity: 5
Size: 0, Capacity: 5


`vector` defines the method `swap`, that swaps the contents of two vectors, in $O(1)$ worse-case time complexity.

In [158]:
#include <iostream>
#include <vector>

template<class T>
void forwardly(const char * const label, std::vector<T> &vec) {
    std::cout << label << ": ";
    typename std::vector<T>::iterator iter = vec.begin();
    while (iter != vec.end()) {
        std::cout << *iter++ << ", ";
    }
    std::cout << "\b\b \n";
}

int main() {
    std::vector<int> v1(5, 1);
    std::vector<int> v2(3, 2);
    
    forwardly("Vector [1]:", v1);
    forwardly("Vector [2]:", v2);
    
    v1.swap(v2);
    std::cout << "After swapping...\n";
    
    forwardly("Vector [1]:", v1);
    forwardly("Vector [2]:", v2);
}

Vector [1]:: 1, 1, 1, 1, 1,  
Vector [2]:: 2, 2, 2,  
After swapping...
Vector [1]:: 2, 2, 2,  
Vector [2]:: 1, 1, 1, 1, 1,  


To compare two `vector` objects, the type of objects the two vectors contain must define the `<` and `==` operators between them.

In [ ]:
std::vector<A> v_a(...);
std::vector<B> v_b(...);

v_a < v_b;                /* 'A' < 'B' must be defined  */

v_a == v_b;               /* 'A' == 'B' must be defined */

v_a != v_b;               /* !('A' == 'B')              */

v_a > v_b;                /* B' < 'A'                   */

v_a <= v_b;               /* !('B' < 'A')               */

v_a >= v_b;               /* !('A' < 'B')               */

*Note:* The implementation of `<` and `==` operators must be `const`.

*Note:* The implementation of `<` and `==` must be transitive.

*Note:* For two `vector` objects, `x` and `y`, `x` is less than `y`: 
* If for the first non-equal elements at index `i`, `x[i] < y[i]` evaluates to true.
* If `x[i] == y[i]` evaluates to true for all elements in `x`, and `x.size() < y.size()` evaluates to true.

`vector` defines the type `value_type`, that is the type of the objects contained within. This can be used to define functions that deal with `vector` objects generically.

In [195]:
#include <iostream>
#include <vector>

template<class T>
typename std::vector<T>::iterator search(std::vector<T> &vec, const typename std::vector<T>::value_type &value) {
    typename std::vector<T>::iterator iter(vec.begin());
    while (iter != vec.end()) {
        if (*iter == value) {
            break;
        }
        iter++;
    }
    return iter;
}

int main() {
    std::vector<int> v(5);
    
    for (int i = 0; i < v.size(); i++) {
        v[i] = i+1;
    }
    
    std::vector<int>::iterator iter = search(v, 5);
    
    if (iter != v.end()) {
        std::cout << "FOUND (" << *iter << ')';
    } else {
        std::cout << "NOT FOUND";
    }
}

FOUND (5)

The `vector` template defines a specialization `vector<bool>` that stores `bool` values as bits, to save memory.

In [205]:
#include <iostream>
#include <vector>
                            /* Capacity rounds up to multiples of an implementation-dependent value */
int main() {
    std::vector<bool> v(1);        
    
    std::cout << v.capacity() << '\n';   
    
    v.reserve(65);
    
    std::cout << v.capacity() << '\n';
}

64
128


#### Summary of `vector` <a class="anchor" id="summary-of-vector"></a>

| *Method* |
| :--- |
| `vector()` |
| `vector(N)` |
| `vector(N, OBJ)` |
| `vector(ITER_BEGIN, ITER_END)` |
| `vector(VECTOR)` |
| ㅤ |
| `assign(N, OBJ)` |
| `assign(ITER_BEGIN, ITER_END)` |
| `=` |
| ㅤ |
| `size()` |
| `resize(N)` |
| `max_size()` |
| `empty()` |
| ㅤ |
| `capacity()` |
| `reserve(N)` |
| ㅤ |
| `[]` |
| `at(INDEX)` |
| `front()` |
| `back()` |
| ㅤ |
| `push_back(OBJ)` |
| `pop_back()` |
| ㅤ |
| `begin()` |
| `end()` |
| `rbegin()` |
| `rend()` |
| ㅤ |
| `insert(ITER_POS, OBJ)` |
| `insert(ITER_POS, N, OBJ)` |
| `insert(ITER_POS, ITER_BEGIN, ITER_END)` |
| ㅤ |
| `erase(ITER_POS)` |
| `erase(ITER_START, ITER_STOP)` |
| `clear()` |
| ㅤ |
| `swap(VECTOR)` |

#### `list` <a class="anchor" id="list"></a>

A `list` is, most frequently, an implementation of a doubly linked list.

*Note:* `list` is optimized for `insert` and `erase` operations.

Compared to `vector`, some methods are missing from `list`.

| *Missing Method* |
| --- |
| `capacity` |
| `reserve` |
| `[]` |
| `at` |

Compared to `vector`, new methods, that push and pop from the front, are added to `list`.

| *Added Method* | *Worst-case Time Complexity* |
| --- | --- |
| `push_front` | $O(1)$ |
| `pop_front` | $O(1)$ |

The maximum size of a `list` is limited only by the data type of the variable tracking the size.

In [207]:
#include <iostream>
#include <list>

int main() {
    std::list<float> ll_float;
    std::list<double> ll_double;
    
    std::cout << ll_float.max_size() << '\n';
    std::cout << ll_double.max_size() << '\n';
}

384307168202282325
384307168202282325


Iterators in `list` are bi-directional. That is, they may be incremented and decremented only.

In [239]:
#include <iostream>
#include <list>

int main() {
    int arr[] = {1, 2, 3, 4, 5};

    std::list<int> ll_int(arr, arr + 5);
    
    std::cout << *ll_int.begin() << '\n';
    std::cout << *++ll_int.begin() << '\n';
}

1
2


`list` also defines the `splice` method, that moves elements from one `list` to another, without copying.

In [ ]:
std::list<int> ls1(...);

std::list<int> ls2(...);

In [ ]:
                                                         /* Moves elements from 'ls2' to 'ls1' */
                                                         /* DEST: Before 'ITER_LS1' */
                                                         
ls1.splice(ITER_LS1, ls2);                               /* SRC: All elements */
                                                        
ls1.splice(ITER_LS1, ls2, ITER_LS2);                     /* SRC: Element at 'ITER_LS2' */

ls1.splice(ITER_LS1, ls2, ITER_LS2_A, ITER_LS2_B);       /* SRC: Elements from 'ITER_LS2_A' to before */
                                                         /*      'ITER_LS2_B'                         */

`list` also defines the `merge` method, that merges elements from one `list` into another, without copying.

* If the two lists are initially sorted, the merged list is sorted.
* If the two lists are unsorted, the merged list is unsorted.

*Note:* The algorithm used to `merge` two sorted lists is stable. That is, it gurantees the same relative order of equal elements before and after merging.

In [249]:
#include <iostream>
#include <list>

template<class T>
void show(const char *label, T cont) {
    std::cout << '[' << label << "]: ";
    typename T::iterator iter(cont.begin());
    while (iter != cont.end()) {
        std::cout << *iter << ", ";
        iter++;
    }
    std::cout << "\b\b \n";
}

int main() {
    int arr[] = {1, 3, 4, 5, 8, 0, 2, 6, 7, 9};

    std::list<int> ls1(arr, arr + 5);
    std::list<int> ls2(arr + 5, arr + 10);
    
    show("List-1", ls1);
    show("List-2", ls2);
    
    std::cout << "\nAfter merging 'ls2' into 'ls1'\n\n";
    
    ls1.merge(ls2);
    
    show("List-1", ls1);
    show("List-2", ls2);
}

[List-1]: 1, 3, 4, 5, 8,  
[List-2]: 0, 2, 6, 7, 9,  

After merging 'ls2' into 'ls1'

[List-1]: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9,  
[List-2]:  


By default, the `merge` method uses the `<` operator to compare elements. Alternatively, it may be passed a *comparator* function object that returns `true` if its first argument should preceed its second argument in ordering.

A *function object* is an object that overloads the function call operator, `()`.

*Note:* A function object is preferred over a function pointer, because the compiler can much more easily inline its call.

In [253]:
#include <iostream>
#include <list>

template<class T>
void show(const char *label, T cont) {
    std::cout << '[' << label << "]: ";
    typename T::iterator iter(cont.begin());
    while (iter != cont.end()) {
        std::cout << *iter << ", ";
        iter++;
    }
    std::cout << "\b\b \n";
}

class Comparator {
public:
    bool operator () (int a, int b) {
        return a < b;
    }
};

int main() {
    int arr[] = {1, 3, 4, 5, 8, 0, 2, 6, 7, 9};

    std::list<int> ls1(arr, arr + 5);
    std::list<int> ls2(arr + 5, arr + 10);
    
    show("List-1", ls1);
    show("List-2", ls2);
    
    std::cout << "\nAfter merging 'ls2' into 'ls1'\n\n";
    
    ls1.merge(ls2, Comparator());
    
    show("List-1", ls1);
    show("List-2", ls2);
}

[List-1]: 1, 3, 4, 5, 8,  
[List-2]: 0, 2, 6, 7, 9,  

After merging 'ls2' into 'ls1'

[List-1]: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9,  
[List-2]:  


`list` also defines the `sort` method.

In [296]:
#include <iostream>
#include <list>

template<class T>
void show(const char *label, T cont) {
    std::cout << '[' << label << "]: ";
    typename T::iterator iter(cont.begin());
    while (iter != cont.end()) {
        std::cout << *iter << "  ";
        iter++;
    }
    std::cout << "\b\b \n";
}

class Comparator {
public:
    bool operator () (int a, int b) {
        return a < b;
    }
};

int main() {
    double arr[] = {1.1, 2.2, 1.3, 1.2, 4.5, 2.1};

    std::list<double> ls(arr, arr + 6);
    
    show("List", ls);
    
    std::cout << "\nAfter sorting...\n\n";
    
    ls.sort(Comparator());                /* sorts, neglecting the fraction part */
                                            /* 'sort()' also possible (doesn't ignore fraction) */
    
    show("List", ls);
}

[List]: 1.1  2.2  1.3  1.2  4.5  2.1   

After sorting...

[List]: 1.1  1.2  1.3  2.1  2.2  4.5   


*Note:* The algorithm used to `sort` a `list` is stable. That is, it gurantees the same relative order of equal elements before and after sorting.

`list` also defines the `remove` method, that removes elements from a list, equal to a passed object, using `==` to test for equality.

In [260]:
#include <iostream>
#include <list>

template<class T>
void show(const char *label, T cont) {
    std::cout << '[' << label << "]: ";
    typename T::iterator iter(cont.begin());
    while (iter != cont.end()) {
        std::cout << *iter << "  ";
        iter++;
    }
    std::cout << "\b\b \n";
}

int main() {
    int arr[] = {1, 2, 2, 3, 3, 3, 4};

    std::list<int> ls(arr, arr + 7);
    
    show("List", ls);
    
    ls.remove(3);
    
    std::cout << "\nAfter removing (3) ...\n\n";
    
    show("List", ls);
}

[List]: 1  2  2  3  3  3  4   

After removing (3)...

[List]: 1  2  2  4   


`list` also defines the `remove_if` method, that accepts a function object instead, that returns `true` if the passed element is to be removed.

In [270]:
#include <iostream>
#include <list>

template<class T>
void show(const char *label, T cont) {
    std::cout << '[' << label << "]: ";
    typename T::iterator iter(cont.begin());
    while (iter != cont.end()) {
        std::cout << *iter << "  ";
        iter++;
    }
    std::cout << "\b\b \n";
}

template<class T>
class Remove_Range {
private:
    T lower, upper;
public:
    Remove_Range(T lower, T upper) : lower(lower), upper(upper) {}

    bool operator () (const T &elem) {
        return elem >= lower && elem <= upper;
    }
};

int main() {
    int arr[] = {1, 2, 3, 4, 5, 6, 7, 8, 9};

    std::list<int> ls(arr, arr + 9);
    
    show("List", ls);
    
    ls.remove_if(Remove_Range<int>(3, 6));
    
    std::cout << "\nAfter removing range (3, 6) all-inclusive ...\n\n";
    
    show("List", ls);
}

[List]: 1  2  3  4  5  6  7  8  9   

After removing range (3, 6) all-inclusive ...

[List]: 1  2  7  8  9   


`list` also defines the `unique` method, that removes repetitions of objects in a `list`.

*Note:* The `unique` method removes only consecutive repetitions of an object.

In [292]:
#include <iostream>
#include <list>

template<class T>
void show(const char *label, T cont) {
    std::cout << '[' << label << "]: ";
    typename T::iterator iter(cont.begin());
    while (iter != cont.end()) {
        std::cout << *iter << "  ";
        iter++;
    }
    std::cout << "\b\b \n";
}

template<class T>
class Equality {
public:
    bool operator () (const T &a, const T &b) {
        return a == b;
    }
};

int main() {
    int arr[] = {1, 1, 2, 2, 3, 3, 3, 4, 5};

    std::list<int> ls1(arr, arr + 9);
    show("List-1", ls1);
    ls1.unique();                                           /* Remove repetitions using '==' operator */
    std::cout << "\nAfter 'unique()'\n\n";
    show("List-1", ls1);
    
    std::cout << "\n--------------------------------------\n\n";
    
    std::list<int> ls2(arr, arr + 9);
    show("List", ls2);
    ls2.unique(Equality<int>());                            /* Remove repetitions using func. obj. */
    std::cout << "\nAfter 'unique(Equality<int>())'\n\n";
    show("List", ls2);
}

[List-1]: 1  1  2  2  3  3  3  4  5   

After 'unique()'

[List-1]: 1  2  3  4  5   

--------------------------------------

[List]: 1  1  2  2  3  3  3  4  5   

After 'unique(Equality<int>())'

[List]: 1  2  3  4  5   


`list` also defines the `reverse` method, that reverses the order of elements in a `list`.

In [295]:
#include <iostream>
#include <list>

template<class T>
void show(const char *label, T cont) {
    std::cout << '[' << label << "]: ";
    typename T::iterator iter(cont.begin());
    while (iter != cont.end()) {
        std::cout << *iter << "  ";
        iter++;
    }
    std::cout << "\b\b \n";
}

int main() {
    int arr[] = {1, 2, 3, 4, 5};

    std::list<int> ls1(arr, arr + 5);
    show("List", ls1);
    ls1.reverse();
    std::cout << "\nAfter 'reverse()'\n\n";
    show("List", ls1);
}

[List]: 1  2  3  4  5   

After 'reverse()'

[List]: 5  4  3  2  1   


#### Summary of `list` <a class="anchor" id="summary-of-list"></a>

| *Method* |
| :--- |
| `list()` |
| `list(N)` |
| `list(N, OBJ)` |
| `list(ITER_BEGIN, ITER_END)` |
| `list(LIST)` |
| ㅤ |
| `assign(N, OBJ)` |
| `assign(ITER_BEGIN, ITER_END)` |
| `=` |
| ㅤ |
| `size()` |
| `resize(N)` |
| `max_size()` |
| `empty()` |
| ㅤ |
| `front()` |
| `back()` |
| ㅤ |
| `push_back(OBJ)` |
| `pop_back()` |
| `push_front(OBJ)` |
| `pop_front()` |
| ㅤ |
| `begin()` |
| `end()` |
| `rbegin()` |
| `rend()` |
| ㅤ |
| `insert(ITER_POS, OBJ)` |
| `insert(ITER_POS, N, OBJ)` |
| `insert(ITER_POS, ITER_BEGIN, ITER_END)` |
| ㅤ |
| `erase(ITER_POS)` |
| `erase(ITER_START, ITER_STOP)` |
| `clear()` |
| ㅤ |
| `swap(LIST)` |
| ㅤ |
| `splice(ITER_POS, LIST)` |
| `splice(ITER_POS, LIST, ITER_LOC)` |
| `splice(ITER_POS, LIST), ITER_BEGIN, ITER_END)` |
| ㅤ |
| `merge(LIST)` |
| `merge(LIST, FUNC_OBJ)` |
| ㅤ |
| `sort()` |
| ㅤ`sort(FUNC_OBJ)` |
| ㅤ |
| `remove(OBJ)` |
| `remove_if(FUNC_OBJ)` |
| ㅤ |
| `unique()` |
| ㅤ`unique(FUNC_OBJ)` |
| ㅤ |
| `reverse()` |

#### `deque` <a class="anchor" id="deque"></a>

A `deque` is a double-ended queue. It supports all operations in a `vector`, in addition to `push_front` and `pop_front`.

It is optimized so that operations at both ends are about as efficient as for a `list`, whereas subscripting approaches the efficiency of a `vector`. 

`insert` and `erase` operations from the middle of a `deque` have `vector`-like inefficiencies.

### Sequence Adapters <a class="anchor" id="sequence-adapters"></a>

A *sequence adapter* is a sequence (container), that relies on another sequence (container) to perform its intended operations.

Within, it defines an instance of the sequence as an attribute, and calls its methods within the bodies of its own.

For all sequence adapters, the sequence it relies on is provided as a secondary template argument, and defaults to `deque`.

In [ ]:
stack<int, vector<int>> s;                     /* `stack` relies on `vector` */

In [ ]:
queue<int> q;                                  /* `queue` relies on `deque` */

In [ ]:
priority_queue<int, list<int>> pq;             /* `priority_queue` relies on `list` */

All sequence adapters may be initialized (or, copy-constructed) from a sequence object, that must match the type of sequence the adapter relies on.

In [ ]:
vector<int> v;

stack<int, vector<int>> s(v);

#### `stack` <a class="anchor" id="stack"></a>

`stack` allows for stack operations only.

| *Method* |
| :--- |
| `stack()` |
| `stack(SEQUENCE)` |
| `stack(STACK)` |
| ㅤ |
| `=` |
| ㅤ |
| `size()` |
| `empty()` |
| ㅤ |
| `top()` |
| ㅤ |
| `push(OBJ)` |
| `pop()` |

#### `queue` <a class="anchor" id="queue"></a>

`queue` allows for queue operations only.

| *Method* |
| :--- |
| `queue()` |
| `queue(SEQUENCE)` |
| `queue(QUEUE)` |
| ㅤ |
| `=` |
| ㅤ |
| `size()` |
| `empty()` |
| ㅤ |
| `front()` |
| `back()` |
| ㅤ |
| `push(OBJ)` |
| `pop()` |

*Note:* A `queue` calls the `pop_front` method and hence, cannot rely on a `vector`.

#### `priority_queue` <a class="anchor" id="priority-queue"></a>

A `priority_queue` is a `queue` in which each element is given a priority that controls the order in which elements get to the `top`.

| *Method* |
| :--- |
| `priority_queue()` |
| `priority_queue(FUNC_OBJ)` |
| `priority_queue(FUNC_OBJ, SEQUENCE)` |
| `priority_queue(ITER_BEGIN, ITER_END, ...)` |
| ㅤ |
| `=` |
| ㅤ |
| `size()` |
| `empty()` |
| ㅤ |
| `top()` |
| ㅤ |
| `push(OBJ)` |
| `pop()` |

*Note:* A `priority_queue` uses the random-access iterators of a sequence and hence, cannot rely on a `list`.

*Note:* An efficient implementation of a `priority_queue` is using a heap data structure, with $O(log(n))$ complexity in `push` and `pop` operations.

By default, `priority_queue` compares elements using `<`, and bubbles larger elements to the top.

A function object's class may be passed as a ternary template argument, and is relied on for comparison instead.

In [333]:
#include <queue>
#include <iostream>
#include <vector>

class Compare {
    char isEven;
public:
    Compare(char isEven=0) : isEven(isEven) {}

    bool operator () (int x, int y) {
        return x % 2 == isEven;                /* bubbles odd or even elements to the top */
    }
};

int main() {
    int arr[] = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10};
    
    std::priority_queue<int, std::vector<int>, Compare> pq(arr, arr + 10, Compare(1));
    
    while (!pq.empty()) {
        std::cout << pq.top() << ' ';
        pq.pop();
    }
}

6 10 2 8 4 3 5 9 1 7 

*Note:* The order of equal elements is undefined in a `priority_queue`.

### Associative Containers <a class="anchor" id="associative-containers"></a>

#### `map` <a class="anchor" id="map"></a>

A `map` is a dictionary of key-value pairs, where each key is associated with a single value.

A common implementation of a `map` is using a *Binary Search Tree* (BST) structure. Keys are stored and retrieved in $O(log(n))$ time complexity.

*Note:* By default, keys are compared using `<`. Equivalence is tested using `!((x < y) || (y < x))`.

*Note:* `map` stores key-value pairs in ascending order.

In [ ]:
std::map<KEY_TYPE, VALUE_TYPE> m;

In [ ]:
std::map<KEY_TYPE, VALUE_TYPE, FUNC_OBJ_CLASS> m;

The `map` class defines three useful data types, within.

| *Alias* | *Type* |
| :-- | :-- |
| `key_type` | `KEY_TYPE`
| `mapped_type` | `VALUE_TYPE`
| `value_type` | `pair<KEY_TYPE, VALUE_TYPE>`

*Note:* `std::pair` is a class in its own right, defined in `<utility>`.

`map` defines a number of constructors.

| *Constructor* |
| :-- |
| `map()` |
| `map(FUNC_OBJ)` |
| `map(ITER_BEGIN, ITER_END, ...)` |
| `map(MAP)` |

`map` defines the `[]` operator for accessing values. 
* If a passed key is not found, it is added and the value is default-initialized.
* If the key already exists, the value is overwritten.

In [339]:
#include <map>
#include <iostream>

int main() {
    std::map<int, double> m;
    
    std::cout << "Size: " << m.size() << '\n';
    std::cout << "Accessing empty map with key '0': " << m[0] << '\n';
    std::cout << "Size: " << m.size() << '\n';
    std::cout << "Setting key '0': " << (m[0] = 5.3) << '\n';
    std::cout << "Size: " << m.size() << '\n';
}

Size: 0
Accessing empty map with key '0': 0
Size: 1
Setting key '0': 5.3
Size: 1


`map` defines bi-directional `iterator` and `reverse_iterator` types, with their respective methods.

In [12]:
#include <map>
#include <iostream>

int main() {
    std::map<int, double> m;
    
    for (int i = 9; i >= 0; i -= 2) {
        m[i] = i - 5.1;
    }
    
    std::map<int, double>::value_type p;
    
    std::map<int, double>::iterator iter = m.begin();
    
    while (iter != m.end()) {
        std::cout << '[' << iter->first << "]: " << iter->second << '\n';
        iter++;
    }
}

[1]: -4.1
[3]: -2.1
[5]: -0.1
[7]: 1.9
[9]: 3.9


`map` defines the `find` method, that returns an `iterator` to a key argument. If not found, `end()` is returned.

In [17]:
#include <map>
#include <iostream>

template<class K, class T, class CMP>
void test(std::map<K, T, CMP> &m, K key) {
    typename std::map<K, T, CMP>::iterator iter = m.find(key);
    
    std::cout << '[' << key << "]: ";

    if (iter == m.end()) {
        std::cout << "NOT FOUND\n";
    } else {
        std::cout << iter->second << '\n';
    }
}

int main() {
    std::map<int, double> m;
    
    for (int i = 9; i >= 0; i -= 2) { m[i] = i - 5.1; }
    
    test(m, 5);
    test(m, 6);
}

[5]: -0.1
[6]: NOT FOUND


`map` defines the `lower_bound` and `upper_bound` methods.
* `lower_bound` returns an iterator to the first pair with key equal to or larger than the passed argument.
* `upper_bound` returns an iterator to the first pair with key larger than the passed argument.

If not found, `end()` is returned.

In [29]:
#include <map>
#include <iostream>

template<class K, class T>
void show_pair(typename std::pair<K, T> &p) {
    std::cout << '[' << p.first << "]: " << p.second << '\n';
}

template<class K, class T, class CMP>
void test(std::map<K, T, CMP> &m, K upper, K lower) {       /* from lower, to upper, all inclusive */
    typedef typename std::map<K, T, CMP>::iterator ITER;
    
    ITER iter_l = m.lower_bound(upper);
    ITER iter_u = m.upper_bound(lower);
    
    while (iter_l != iter_u) {
        show_pair(*(iter_l++));
    }
}

int main() {
    std::map<int, double> m;
    
    for (int i = 0; i < 10; i++) { m[i] = i + 0.5; }
    
    test(m, 5, 9);
}

[5]: 5.5
[6]: 6.5
[7]: 7.5
[8]: 8.5
[9]: 9.5


`map` defines the `insert` method, that accepts `value_type` and returns `pair<iterator, bool>`. If key already exists, nothing happens, and `bool` is `false`.

In [38]:
#include <map>
#include <iostream>

template<class K, class T, class CMP>
void insert(std::map<K, T, CMP> &m, K key, T value) {
    typedef typename std::map<K, T, CMP>::iterator ITER;
    
    std::pair<ITER, bool> res = m.insert(std::pair<K, T>(key, value));
    
    if (res.second) {
        std::cout << "INSERTED SUCCESSFULLY!\n";
    } else {
        std::cout << "FAILED INSERTION.\n";
    }
}

int main() {
    std::map<int, double> m;
    
    insert(m, 0, 5.3);
    insert(m, 0, 6.7);
}

INSERTED SUCCESSFULLY.
FAILED INSERTION.


*Note:* Unlike `[]`, `insert` does not overwrite the value of an already existing key.

`map` overloads the `insert` method, with two useful implementations.

* `insert(ITER_POS, PAIR)` : Inserts `PAIR`, uses `ITER_POS` as an optimizing hint, returns an `iterator` to the new or already existing key.

* `insert(ITER_BEGIN, ITER_END)` : Inserts all elements, which must be of type `pair<KEY_TYPE, VALUE_TYPE>`, returns `void`.

*Note:* Unlike the unoptimized single element implementation, the latter does not report the status of the insertion operation.

`map` also defines the `erase` method:

* `erase(ITER_POS)` : Erases pair at `ITER_POS`, returns `void`.
* `erase(KEY)` : Erases pair with `KEY`, returns `1` if found and `0` otherwise.
* `erase(ITER_START, ITER_END)` : Erases pair from `ITER_START`, inclusive, to `ITER_END`, exclusive. 

In [11]:
#include <map>
#include <iostream>

template<class T>
void show(const char *label, T &c) {
    std::cout << label << '\n';
    
    typename T::iterator iter = c.begin();
    
    while (iter != c.end()) {
        std::cout << "  [" << iter->first << "]: " << iter->second << '\n';
        iter++;
    }
}

int main() {
    std::map<int, double> m;
    
    for (int i = 0; i < 5; i++) { m[i] = i + 0.5; }
    
    show("Before", m);
    
    m.erase(m.find(1), m.find(4));
    
    show("After", m);
}

Before
  [0]: 0.5
  [1]: 1.5
  [2]: 2.5
  [3]: 3.5
  [4]: 4.5
After
  [0]: 0.5
  [4]: 4.5


*Note:* Calling the `erase` method on `end()` is harmless.

#### Summary of `map` <a class="anchor" id="summary-of-map"></a>

| *Method* |
| :--- |
| `map()` |
| `map(FUNC_OBJ)` |
| `map(ITER_BEGIN, ITER_END, ...)` |
| `map(MAP)` |
| ㅤ |
| `=` |
| ㅤ |
| `size()` |
| `max_size()` |
| `empty()` |
| ㅤ |
| `begin()` |
| `end()` |
| `rbegin()` |
| `rend()` |
| ㅤ |
| `[]` |
| ㅤ |
| `find(KEY)` |
| `lower_bound(KEY)` |
| `upper_bound(KEY)` |
| ㅤ |
| `insert(PAIR)` |
| `insert(ITER_POS, PAIR)` |
| `insert(ITER_BEGIN, ITER_END)` |
| ㅤ |
| `erase(KEY)` |
| `erase(ITER_POS)` |
| `erase(ITER_START, ITER_END)` |
| `clear()` |

#### `multimap` <a class="anchor" id="multimap"></a>

A `multimap` is a `map` with more than one value possible for each key.

In `multimap`, the `[]` operator is undefined, and `insert` method returns `iterator` and not `pair<iterator, bool>`, naturally.

`multimap` defines the `equal_range` method, that accepts a key and returns `pair<iterator, iterator>`, with each iterator corresponding to `lower_bound` and `upper_bound`, respectively.

In [29]:
#include <map>
#include <iostream>

template<class K, class T>
void show_pair(typename std::pair<K, T> &p) {
    std::cout << '[' << p.first << "]: " << p.second << '\n';
}

template<class K, class T, class CMP>
void test(std::multimap<K, T, CMP> &m, K key) {
    typedef typename std::multimap<K, T, CMP>::iterator ITER;
    
    std::pair<ITER, ITER> res = m.equal_range(key);
    
    while (res.first != res.second) {
        show_pair(*res.first);
        res.first++;
    }
}

int main() {
    std::multimap<int, double> m;
    
    for (int i = 0; i < 3; i++) { 
        m.insert(std::pair<int, double>(i, i + 0.2));
        m.insert(std::pair<int, double>(i, i + 0.5));
        m.insert(std::pair<int, double>(i, i + 0.7));
    }
    
    test(m, 1);
}

[1]: 1.2
[1]: 1.5
[1]: 1.7


*Note:* The `equal_range` method is also defined in `map`, for compatibility of templated functions dealing with both `map` and `multimap`.

*Note:* The `find` method in `multimap` returns an iterator to the first matching key-value pair, ascendingly.

`multimap` also defines the `count` method, that returns the number of values for a passed key argument.

In [36]:
#include <map>
#include <iostream>

int main() {
    std::multimap<int, double> m;
    
    for (int i = 0; i < 3; i++) { 
        m.insert(std::pair<int, double>(i, i + 0.2));
        m.insert(std::pair<int, double>(i, i + 0.5));
        m.insert(std::pair<int, double>(i, i + 0.7));
    }
    
    std::cout << "Size:      " << m.size() << '\n';
    std::cout << "Count(0):  " << m.count(0) << '\n';
    std::cout << "Count(3):  " << m.count(3) << '\n';
}

Size:      9
Count(0):  3
Count(3):  0


#### `set` <a class="anchor" id="set"></a>

A `set` can be seen as a `map` where values are irrelevant, so only keys are kept track off.

It is primarily used to maintain a collection of unique items, avoiding repetitions.

In [ ]:
std::set<TYPE> s;

In [ ]:
std::set<TYPE, FUNC_OBJ_CLASS> s;

In `set`, the `[]` operator is undefined, and `value_type` is defined as `TYPE`.

#### `multiset` <a class="anchor" id="multiset"></a>

Similar to `multimap`, a `multiset` is a `set` that allows duplicates.

It is primarily used to maintain a collection of sorted items.

## Algorithms and Function Objects <a class="anchor" id="algorithms-and-function-objects"></a>

The standard library provides sixty general algorithms, along with useful function object classes.

### Function Objects <a class="anchor" id="function-objects"></a>

All function object classes in the standard library are derived from either `unary_function` and `binary_function` templated classes.

In [ ]:
template<class Arg, class Res>
struct unary_function {
    typedef Arg argument_type;
    typedef Res result_type;
};

In [ ]:
template<class Arg, class Arg2, class Res>
struct binary_function {
    typedef Arg first_argument_type;
    typedef Arg2 second_argument_type;
    typedef Res result_type;
};

#### Predicates <a class="anchor" id="predicates"></a>

A *predicate* is a function object, or *functor*, that returns a `bool`.

All predicates are of the form `functor-name<ARG-TYPE>`, implying the same type for any number of arguments.

| *Functor* | *Type*
| :--- | --- |
| `equal_to` | Binary
| `not_equal_to` | Binary
| ㅤ |
| `greater` | Binary
| `greater_equal` | Binary
| ㅤ |
| `less` | Binary
| `less_equal` | Binary
| ㅤ |
| `logical_and` | Binary
| `logical_or` | Binary
| `logical_not` | Unnary

#### Arithmetic Functors <a class="anchor" id="arithmetic-functors"></a>

An arithmetic functor is a functor that returns the result of a built-in arithmetic operator.

All arithmetic functors are of the form `functor-name<ARG-TYPE>`, implying the same type for any number of arguments.

| *Functor* | *Type*
| :--- | --- |
| `plus` | Binary
| `minus` | Binary
| ㅤ |
| `multiplies` | Binary
| `divides` | Binary
| `modulus` | Binary
| ㅤ |
| `negate` | Unary

#### Binders <a class="anchor" id="binders"></a>

A *binder* is a functor that accepts another binary functor and an argument. 

It binds the first or second parameter of the binary functor to the passed argument.

In [ ]:
    // Example: 'binder2nd' binds the 2nd argument of a binary functor

template<class BinOp>
struct binder2nd : public unary_function<BinOp::first_argument_type, BinOp::result_type> {
protected:
    BinOp op;
    typename BinOp::second_argument_type arg2;
public:
    binder2nd(const BinOp &op, const typename BinOp::second_argument_type &arg2)
        : op(op), arg2(arg2) {}
    
    result_type operator() (const argument_type &x) const { 
        return op(x, arg2);
    }
};

A binder class is accompained by a function that returns a binder functor from passed arguments.

In [21]:
#include <functional>
#include <iostream>

int main() {
    std::cout << std::less<int>()(5, 6);
    std::cout << std::less<int>()(7, 6);
    
        // Example: 'bind2nd' accompanies 'binder2nd', both defined in '<functional>'
    
    std::cout << std::bind2nd(std::less<int>(), 6)(5);
    std::cout << std::bind2nd(std::less<int>(), 6)(7);
}

1010

Alternatively, a new functor class may be defined, that inherits from the binder class.

In [25]:
//%cflags: -w

#include <functional>
#include <iostream>

template<class T>
struct less_than : public std::binder2nd<std::less<int>> {
    less_than(T arg) : binder2nd(std::less<int>(), arg) {}
};

int main() {
    less_than<int> fobj(6);

    std::cout << fobj(5);
    std::cout << fobj(7);
}

10

*Note:* The standard library provides two binders: `binder1st` and `binder2nd`. 

#### Function Adapters <a class="anchor" id="function-adapters"></a>

A function adapter is a functor that calls another function implicitly.

A pointer-to-member function adapter is returned by `mem_fun`.

The returned functor expects a pointer to an object, from the appropriate class, as argument.

In [47]:
#include <iostream>
#include <functional>
#include <vector>

class Complex {
    double real, imag;
public:
    Complex(double r, double i) : real(r), imag(i) {}
    
    void print(const char *str) {
        std::cout << str << "[r]: " << real << ", [i]: " << imag << '\n';
    }
};

int main() {
    std::vector<Complex *> v;
    
    v.reserve(5);
    
    for (int i = 1; i <= 3; i++) {
        v.push_back(new Complex(i + 0.1*i, 3 - i + 0.2*i));
    }
    
    for_each(v.begin(), v.end(), std::bind2nd(std::mem_fun(&Complex::print), "Complex - "));
}

Complex - [r]: 1.1, [i]: 2.2
Complex - [r]: 2.2, [i]: 1.4
Complex - [r]: 3.3, [i]: 0.6


A pointer-to-member function adapter is also returned by `mem_fun_ref`.

The returned functor expects a reference to an object, from the appropriate class, as argument.

In [43]:
#include <iostream>
#include <functional>
#include <vector>

class Complex {
    double real, imag;
public:
    Complex(double r, double i) : real(r), imag(i) {}
    
    void print() {
        std::cout << "[r]: " << real << ", [i]: " << imag << '\n';
    }
};

int main() {
    std::vector<Complex> v;
    
    v.reserve(5);
    
    for (int i = 1; i <= 3; i++) {
        v.push_back(Complex(i + 0.1*i, 3 - i + 0.2*i));
    }
    
    for_each(v.begin(), v.end(), std::mem_fun_ref(&Complex::print));
}

[r]: 1.1, [i]: 2.2
[r]: 2.2, [i]: 1.4
[r]: 3.3, [i]: 0.6


A function pointer adapter is returned by `ptr_fun`.

The returned functor expects a reference to an object, from the appropriate class, as argument.

In [48]:
#include <iostream>
#include <functional>
#include <vector>

class Complex {
    double real, imag;
public:
    Complex(double r, double i) : real(r), imag(i) {}
    
    friend void print(const Complex &c);
};

void print(const Complex &c) {
    std::cout << "[r]: " << c.real << ", [i]: " << c.imag << '\n';
}

int main() {
    std::vector<Complex> v;
    
    v.reserve(5);
    
    for (int i = 1; i <= 3; i++) {
        v.push_back(Complex(i + 0.1*i, 3 - i + 0.2*i));
    }
    
    for_each(v.begin(), v.end(), std::ptr_fun(&print));
}

[r]: 1.1, [i]: 2.2
[r]: 2.2, [i]: 1.4
[r]: 3.3, [i]: 0.6


*Note:* Function adapters are able to deal with up to two arguments.

#### Negaters <a class="anchor" id="negaters"></a>

A *negater* is a functor that implements the logical not of a passed predicate argument.

In [58]:
#include <iostream>
#include <functional>

template<class T>
void test_binary(T fobj) {
    std::cout << fobj(false, false);
    std::cout << fobj(false, true);
    std::cout << fobj(true, false);
    std::cout << fobj(true, true);
    std::cout << '\n';
}

template<class T>
void test_unary(T fobj) {
    std::cout << fobj(false);
    std::cout << fobj(true);
    std::cout << '\n';
}

int main() {
    test_binary(std::logical_and<bool>());
    test_binary(not2(std::logical_and<bool>()));    // 'not2' : negates a binary predicate
    
    test_unary(std::logical_not<bool>());
    test_unary(not1(std::logical_not<bool>()));     // 'not1' : negates a unary predicate
}

0001
1110
10
01


### Algorithms <a class="anchor" id="algorithms"></a>

#### Non-modifying Sequence Operations <a class="anchor" id="non-modifying-sequence-operations"></a>

`for_each` performs an operation for each element in a sequence.

In [102]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

struct inc {
    void operator() (int &val) { val++; }
};

int main() {
    int x[] = {1, 2, 3, 4, 5};
    
    std::vector<int> v(x, x+5);
    
    forwardly(v.begin(), v.end());
    
    for_each(v.begin(), v.end(), inc());
    
    forwardly(v.begin(), v.end());
}

1, 2, 3, 4, 5,   
2, 3, 4, 5, 6,   


*Note:* `for_each` may modify the elements of a sequence, if the passed function object so does.

`find` finds the first occurence of a value in a sequence.

In [104]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int x[] = {1, 2, 3, 4, 5};
    
    int *ptr = std::find(x, x + 5, 3);
    
    if (x + 2 == ptr) {
        std::cout << "Works!";
    }
}

Works!

*Note:* Similar to many functions in the standard library, `find` returns the `end()` iterator if not found.

*Note:* To find the last occurence of a value in a sequence, use reverse iterators. This approach applies to many algorithms in the standard library.

`find_if` finds the first occurence in a sequence, that satisfies a unary predicate.

In [106]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int x[] = {1, 2, 3, 4, 5};
    
    int *ptr = std::find_if(x, x + 5, std::bind2nd(std::greater<int>(), 3));
    
    if (ptr != x + 5) {
        std::cout << "Found: " << *ptr;
    }
}

Found: 4

`find_first_of` finds the first occurence in a sequence, that is found in another sequence.

In [108]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int x[] = {1, 2, 3, 4, 5};
    int y[] = {3, 4, 5, 6, 7};
    
    int *ptr = std::find_first_of(x, x+5, y, y+5);
    
    if (ptr == x + 2) {
        std::cout << "Found: " << *ptr;
    }
}

Found: 3

*Note:* `find_first_of` accepts an optional binary predicate argument.

`adjacent_find` finds the first pair of adjacent matching values.

In [110]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int x[] = {1, 2, 3, 3, 4, 5};
    
    int *ptr = std::adjacent_find(x, x+6);
    
    if (ptr == x + 2) {
        std::cout << "Found: " << *ptr;
    }
}

Found: 3

*Note:* `adjacent_find` accepts an optional binary predicate argument.

`count` returns the number of occurences of a value in a sequence.

In [115]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int x[] = {1, 2, 3, 3, 3, 3, 4, 4, 5, 5};
    
    typedef typename std::iterator_traits<int *>::difference_type u_count;
    
    u_count cnt = std::count(x, x+10, 3);
    
    std::cout << cnt;
}

4

The maximum value returned by `count` is the maximum difference between two iterators of the type given.

This can be retrieved through `typename std::iterator_traits<ITER_TYPE>::difference_type`. This syntax is pointer-compatible, unlike typing `typename ITER_TYPE::difference_type` directly.

`count_if` returns the number of occurences in a sequence, that satisfy a unary predicate.

In [121]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int x[] = {1, 6, 2, 7, 3, 8, 9, 0, 4, 5};
    
    typedef typename std::iterator_traits<int *>::difference_type u_count;
    
    u_count cnt = std::count_if(x, x+10, std::bind2nd(std::greater_equal<int>(), 5));
    
    std::cout << cnt;
}

5

`equal` returns `true` if the elements of two sequences match, and `false` otherwise.

In [124]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int x[] = {1, 6, 2, 7, 3, 8, 9, 0, 4, 5};
    int y[] = {1, 6, 2, 7, 3, 0};
    
    std::cout << std::equal(x, x+5, y);
    std::cout << std::equal(x, x+6, y);
}

10

*Note:* As with many algorithms in the standard library, the second sequence is assumed to have at least the same number of elements as the first.

*Note:* `equal` accepts an optional binary predicate argument.

`mismatch` returns the first pair of iterators of the first non-matching elements, of two passed sequences.

In [126]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int x[] = {1, 6, 2, 7, 3, 1, 1, 1, 1, 1};
    int y[] = {1, 6, 2, 7, 3, 0, 0, 0, 0, 0};
    
    std::pair<int *, int *> pair1 = std::mismatch(x, x+10, y);
    std::pair<int *, int *> pair2 = std::mismatch(x, x+10, x);
    
    if (pair1.first == x+5) { std::cout << "Found a mismatch!\n"; }
    if (pair2.first == x+10) { std::cout << "No-mismatch.\n"; }
}

Found a mismatch!
No-mismatch.


*Note:* `mismatch` accepts an optional binary predicate argument.

`search` checks whether a sub-sequence exists in a sequence.

If found, an iterator to the first element of the sub-sequence, in the sequence, is returned.

In [127]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int x[] = {0, 2, 3, 4, 5, 6, 7, 8, 9};
    int y[] = {3, 4, 5};
    
    int *ptr = std::search(x, x+10, y, y+3);
    
    if (ptr == x+2) {
        std::cout << "Works!";
    }
}

Works!

*Note:* `search` accepts an optional binary predicate argument.

`find_end` is similar to `search`, except it finds the last occurence of the sub-sequence, instead of the first.

In [131]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int x[] = {0, 2, 3, 4, 5, 0, 3, 4, 5, 9};
    int y[] = {3, 4, 5};
    
    int *ptr = std::find_end(x, x+10, y, y+3);
    
    if (ptr == x+6) {
        std::cout << "Works!";
    }
}

Works!

*Note:* `find_end` accepts an optional binary predicate argument.

`search_n` finds the first `n` consequent occurences of a value in a sequence, and returns an iterator to the first element.

In [148]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int x[] = {0, 2, 3, 3, 4, 0, 3, 4, 5, 9};
    
    int *ptr = std::search_n(x, x+10, 2, 3);       /* 2 consequent occurences of 3 */
    
    forwardly(ptr, x+10);
}

3, 3, 4, 0, 3, 4, 5, 9,   


*Note:* `search_n` accepts an optional binary predicate argument.

#### Modifying Sequence Operations <a class="anchor" id="modifying-sequence-operations"></a>

`copy` and `copy_backward` copy the elements of one sequence into another.

In [169]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int x[] = {1, 2, 3, 4, 5, 6, 7, 8, 9};
    int z[] = {1, 3, 5, 7, 9, 7, 5, 3, 1};
    int y[9] = {0};
    
    forwardly(y, y+9);
    
    std::copy(x, x+9, y);                   /* Uses 'begin' iterator */
    
    forwardly(y, y+9);
    
    std::copy_backward(z, z+9, y+9);        /* Does not reverse, but uses 'end' iterator */
    
    forwardly(y, y+9);
}

0, 0, 0, 0, 0, 0, 0, 0, 0  
1, 2, 3, 4, 5, 6, 7, 8, 9  
1, 3, 5, 7, 9, 7, 5, 3, 1  


*Note:* `copy_backward` demands bi-directional iterators as arguments, while `copy` does not.

`transform` copies the elements of one sequence into another, *transforming* each value in the midst.

In [173]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

template<class T>
struct Squared {
    T operator() (T val) {
        return val * val;
    }
};

int main() {
    int x[] = {1, 2, 3, 4, 5, 6, 7, 8, 9};
    int y[] = {9, 8, 7, 6, 5, 4, 3, 2, 1};
    int z[9];
    
    std::transform(x, x+9, z, Squared<int>());            /* One-list Transformation */
                                                          /* Unary Function Object */ 
    forwardly(z, z+9);
    
    std::transform(x, x+9, y, z, std::multiplies<int>());     /* Two-list Transformation */
                                                              /* Binary Function Object */
    forwardly(z, z+9);
}

1, 4, 9, 16, 25, 36, 49, 64, 81,   
9, 16, 21, 24, 25, 24, 21, 16, 9,   


`unique` eliminates adjacent duplicates from a sequence, by moving them to the end of the sequence.

An iterator to the end of the unique subsequence is returned.

In [175]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int y[] = {1, 1, 6, 6, 6, 3, 3, 4, 0};
    
    int *ptr = std::unique(y, y+9);
    
    forwardly(y, ptr);
}

1, 6, 3, 4, 0,   


*Note:* `unique_copy` is similar to `unique`, except it copies unique elements into another sequence. Similarly, it returns an iterator to the end of the unique subsequence.

*Note:* Both `unique` and `unique_copy` accept an optional binary predicate argument.

`replace` replaces matching values of a sequence with another value.

`replace_if` applies a unary predicate instead, to determine whether to replace an element.

`replace_copy` and `replace_copy_if` perform the copy-and-replace operation into another sequence.

In [181]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int y[] = {1, 2, 3, 4, 1, 2, 3, 4};
    int z[8];
    
    std::replace_copy(y, y+8, z, 2, 0);
    
    forwardly(z, z+8);
    
    std::replace_copy_if(y, y+8, z, std::bind2nd(std::less<int>(), 3), 0);
    
    forwardly(z, z+8);
}

1, 0, 3, 4, 1, 0, 3, 4,   
0, 0, 3, 4, 0, 0, 3, 4,   


`remove` removes matching values of a sequence, by moving them to the end of the sequence.

An iterator to the end of the subsequence is returned.

`remove_if` applies a unary predicate instead, to determine whether to remove an element.

`remove_copy` and `remove_copy_if` copy into another sequence.

In [183]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int y[] = {1, 2, 3, 4, 1, 2, 3, 4};
    int z[8];
    int *ptr;
    
    ptr = std::remove_copy(y, y+8, z, 2);
    
    forwardly(z, ptr);
    
    ptr = std::remove_copy_if(y, y+8, z, std::bind2nd(std::less<int>(), 3));
    
    forwardly(z, ptr);
}

1, 3, 4, 1, 3, 4,   
3, 4, 3, 4,   


`fill` assigns a value to each element of a sequence.

`fill_n` assigns a value to the first `n` elements of a sequence.

`generate` and `generate_n` call a function object, called a *generator*, and assign its return value instead.

In [197]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

class Counter {
    int i;
public:
    int operator() () {
        return i++;
    }
};

int main() {
    int y[8] = {0};
    
    std::generate(y, y+4, Counter());
    
    forwardly(y, y+8);
   
    std::fill_n(y+4, 4, 1);
    
    forwardly(y, y+8);
}

0, 1, 2, 3, 0, 0, 0, 0,   
0, 1, 2, 3, 1, 1, 1, 1,   


`reverse` reverses a sequence in place.

`reverse_copy` reverse a sequence into another sequence.

In [202]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

class Counter {
    int i;
public:
    int operator() () {
        return i++;
    }
};

int main() {
    int y[] = {1, 2, 3, 4, 5, 6, 7, 8, 9};
    int z[5];
    
    std::reverse(y, y+5);
    
    forwardly(y, y+9);
    
    std::reverse_copy(y, y+5, z);
    
    forwardly(z, z+5);
}

5, 4, 3, 2, 1, 6, 7, 8, 9,   
1, 2, 3, 4, 5,   


`rotate` rotates a sequence, so that the `middle` element becomes the `first`.

`rotate_copy` rotates a sequence into another sequence.

In [204]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int y[] = {1, 2, 3, 4, 5, 6, 7, 8, 9};
    
    std::rotate(y, y+2, y+9);                /* 'rotate(first, middle, last)'' */
    
    forwardly(y, y+9);
}

3, 4, 5, 6, 7, 8, 9, 1, 2,   


`random_shuffle` shuffles a sequence randomly.

In [205]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int y[] = {1, 2, 3, 4, 5, 6, 7, 8, 9};
    
    std::random_shuffle(y, y+9);
    
    forwardly(y, y+9);
}

5, 4, 8, 9, 1, 6, 3, 2, 7,   


`swap_ranges` swaps the elements of two sequences.

In [211]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int x[] = {1, 1, 1, 1, 1};
    int y[5] = {0};
    
    std::swap_ranges(x+1, x+4, y+1);
    
    forwardly(x, x+5);
    forwardly(y, y+5);
}

1, 0, 0, 0, 1,   
0, 1, 1, 1, 0,   


#### Sorting Sequence Operations <a class="anchor" id="sorting-sequence-operations"></a>

`sort` sorts a sequence, without preserving the relative order of equal elements, unlike `stable_sort`.

In [217]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int x[] = {1, 2, 3, 4, 5, 6, 7, 8, 9};
    
    std::random_shuffle(x, x+9);
    
    forwardly(x, x+9);
    
    std::sort(x, x+9);
    
    forwardly(x, x+9);
}

5, 4, 8, 9, 1, 6, 3, 2, 7,   
1, 2, 3, 4, 5, 6, 7, 8, 9,   


*Note:* By default, sorting algorithms in the standard library use `std::less` to sort, which uses the `<` operator, and they accept an optional binary predicate argument.

*Note:* Sorting algorithms in the standard library require random-access iterators.

`partial_sort` sorts a sequence partially:
* Placing the right elements in `first` to `middle` iterators.
* Placing the remaining elements in `middle + 1` to `last`, unsorted.

In [220]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int x[] = {1, 2, 3, 4, 5, 6, 7, 8, 9};
    
    std::reverse(x, x+9);
    
    forwardly(x, x+9);
    
    std::partial_sort(x, x+5, x+9);
    
    forwardly(x, x+9);
}

9, 8, 7, 6, 5, 4, 3, 2, 1,   
1, 2, 3, 4, 5, 9, 8, 7, 6,   


`partial_sort_copy` sorts a sequence partially into another.

In [223]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int x[] = {1, 2, 3, 4, 5, 6, 7, 8, 9};
    int z[9];
    
    std::random_shuffle(x, x+9);
    
    forwardly(x, x+9);

    std::partial_sort_copy(x, x+9, z, z+5);    /* 'partial_sort(TO_SORT, INTO_SORTED)' */
    
    forwardly(z, z+5);
}

5, 4, 8, 9, 1, 6, 3, 2, 7,   
1, 2, 3, 4, 5,   


`nth_element` sorts a sequence only as far as is necessary to get the `nth` element to its proper place.

In [225]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int x[] = {1, 2, 3, 4, 5, 6, 7, 8, 9};
    
    std::random_shuffle(x, x+9);
    
    forwardly(x, x+9);

    std::nth_element(x, x+4, x+9);       /* 'nth_element(first, nth, last)' */
    
    forwardly(x, x+9);
}

5, 4, 8, 9, 1, 6, 3, 2, 7,   
4, 2, 3, 1, 5, 6, 7, 8, 9,   


`binary_search` searches for a value in a sorted sequence, and returns `bool`.

In [226]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int x[] = {1, 2, 3, 4, 5, 6, 7, 8, 9};

    std::cout << std::binary_search(x, x+9, 5);
    std::cout << std::binary_search(x, x+9, 0);
} 

10

*Note:* `lower_bound`, `upper_bound` and `equal_range` are also provided, implemented similar to their implementation in `std::map`.

`merge` merges two sorted sequences into another sequence.

`inplace_merge` merges two sorted and subsequent subsequences in the same sequence.

In [227]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int x[] = {0, 2, 4, 6, 8, 1, 3, 5, 7, 9};
    int y[10];

    std::merge(x, x+5, x+5, x+10, y);
    
    forwardly(y, y+10);
    
    std::inplace_merge(x, x+5, x+10);
    
    forwardly(x, x+10);
}

0, 1, 2, 3, 4, 5, 6, 7, 8, 9,   
0, 1, 2, 3, 4, 5, 6, 7, 8, 9,   


`parition` partitions elements in a sequence using a unary predicate argument.

`stable_partition` preserves the relative order of elements after partitioning.

In [229]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

template<class T>
struct Even {
    bool operator() (const T &val) {
        return val % 2 == 0;
    }
};

int main() {
    int x[] = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9};

    std::stable_partition(x, x+10, Even<int>());          /* 'True', then 'False' */
    
    forwardly(x, x+10);
}

0, 2, 4, 6, 8, 1, 3, 5, 7, 9,   


#### Set Operations <a class="anchor" id="set-operations"></a>

`includes` checks whether every element in the second set is also in the first set.

In [234]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int x[] = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9};
    int test1[] = {5, 7};
    int test2[] = {-1, 4, 5};
 
    std::cout << std::includes(x, x + 10, test1, test1 + 2);
    std::cout << std::includes(x, x + 10, test2, test2 + 3);
}

10

*Note:* `includes`, as with all set operations, accepts an optional binary predicate argument.

`set_union`, `set_intersection`, `set_difference` and `set_symmetric_difference` perform common set operations.

In [241]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int a1[] = {1, 2, 3, 4};
    int a2[] = {3, 4, 5, 6};
    int res[4];
    int *ptr;
    
    ptr = std::set_union(a1, a1 + 4, a2, a2 + 4, res);
    forwardly(res, ptr);

    ptr = std::set_intersection(a1, a1 + 4, a2, a2 + 4, res);
    forwardly(res, ptr);
    
    ptr = std::set_difference(a1, a1 + 4, a2, a2 + 4, res);
    forwardly(res, ptr);
    
    ptr = std::set_symmetric_difference(a1, a1 + 4, a2, a2 + 4, res);
    forwardly(res, ptr);
}

1, 2, 3, 4, 5, 6,   
3, 4,   
1, 2,   
1, 2, 5, 6,   


#### Heap Operations <a class="anchor" id="heap-operations"></a>

A *heap* is a complete binary tree, where every node must be greater (or, less) than its children.

`make_heap` converts a sequence into a heap, by re-ordering the elements accordingly.

`sort_heap` converts a heap into a sorted sequence.

*Note:* All heap operations accept an optional binary predicate argument.

In [243]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

template<class ITER>
void print_heap(ITER first, ITER last) {
    while (first != last) {
        std::cout << *first++ << ", ";
        pop
    }
    std::cout << "\b\b  \n";
}

int main() {
    int x[] = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9};
    
    std::make_heap(x, x+10);                /* Sequence to Heap */
    forwardly(x, x+10);
    
    for (int i = 0; i < 10; i++) {
        
        std::pop_heap();
    }
    
    std::sort_heap(x, x+10);                /* Heap to Sorted Sequence */
    forwardly(x, x+10);
}

9, 8, 6, 7, 4, 5, 2, 0, 3, 1,   
0, 1, 2, 3, 4, 5, 6, 7, 8, 9,   


`push_heap` assumes the heap spans `first` to `last - 1` before pushing.

Similarly, `pop_heap` returns the heap spanning `first` to `last - 1`.

In [248]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int x[] = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9};
    
    std::make_heap(x, x+10);
    forwardly(x, x+10);
    
    for (int i = 0; i < 9; i++) {
        std::pop_heap(x, x+10-i);
        forwardly(x, x+10-i-1);
    }
}

9, 8, 6, 7, 4, 5, 2, 0, 3, 1,   
8, 7, 6, 3, 4, 5, 2, 0, 1,   
7, 4, 6, 3, 1, 5, 2, 0,   
6, 4, 5, 3, 1, 0, 2,   
5, 4, 2, 3, 1, 0,   
4, 3, 2, 0, 1,   
3, 1, 2, 0,   
2, 1, 0,   
1, 0,   
0,   


#### Permutations <a class="anchor" id="permutations"></a>

`next_permutation` and `prev_permutation` are used to run through all possible permutations of elements in a sequence:
* The sequence is shuffled into the next or previous permutation with every function call.
* Permutations are generated in lexicographical order, using `std::less` by default, accepting an optional binary predicate argument.
* `bool` is returned to indicate whether the next (or, previous) permutation exists.

In [252]:
//%cflags: -I.jupyter

#include <algorithm>
#include <functional>
#include <iostream>
#include <vector>
#include "helpers.h"

int main() {
    int x[] = {1, 2, 3};
    forwardly(x, x+3);
    
    while (std::next_permutation(x, x+3)) {
        forwardly(x, x+3);
    }
}

1, 2, 3,   
1, 3, 2,   
2, 1, 3,   
2, 3, 1,   
3, 1, 2,   
3, 2, 1,   
